# Content Based Recommendation

In [4]:
from zipfile import ZipFile
file_name = 'movies_data.zip'
with ZipFile(file_name,'r') as zip:
    zip.printdir()
    zip.extractall()

File Name                                             Modified             Size
ml-latest-small/                               2018-09-26 15:50:12            0
ml-latest-small/links.csv                      2018-09-26 15:50:10       197979
ml-latest-small/tags.csv                       2018-09-26 15:49:40       118660
ml-latest-small/ratings.csv                    2018-09-26 15:49:38      2483723
ml-latest-small/README.txt                     2018-09-26 15:50:12         8342
ml-latest-small/movies.csv                     2018-09-26 15:49:56       494431


In [11]:
import pandas as pd
import numpy as np
import zipfile

zf = zipfile.ZipFile('movies_data.zip') 


In [56]:
movies_df = pd.read_csv(zf.open('ml-latest-small/movies.csv'))
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [19]:
len(movies_df)

9742

In [21]:
movies_df.tail()

,movieId,title,genres
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation
9741,193609,Andrew Dice Clay: Dice Rules (1991),Comedy


In [57]:
ratings_df = pd.read_csv(zf.open('ml-latest-small/ratings.csv'))
ratings_df.head(10)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


In [18]:
len(ratings_df)

100836

In [20]:
ratings_df.tail()

,userId,movieId,rating,timestamp
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352
100835,610,170875,3.0,1493846415


In [55]:
# using regular expression to extract the year i.e. (xxxx)
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies_df['year'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: year, dtype: object

In [44]:
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,(1995)
1,2,Jumanji (1995),Adventure|Children|Fantasy,(1995)
2,3,Grumpier Old Men (1995),Comedy|Romance,(1995)
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,(1995)
4,5,Father of the Bride Part II (1995),Comedy,(1995)


In [46]:
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
movies_df['year'].head()

0    1995
1    1995
2    1995
3    1995
4    1995
Name: year, dtype: object

In [59]:
#removing year from title column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))','')
#strip whitespaces
movies_df['title'] = movies_df['title'].apply(lambda x : x.strip())
movies_df.head()

,movieId,title,genres
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,Adventure|Children|Fantasy
2,3,Grumpier Old Men,Comedy|Romance
3,4,Waiting to Exhale,Comedy|Drama|Romance
4,5,Father of the Bride Part II,Comedy


In [60]:
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji,"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men,"[Comedy, Romance]"
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II,[Comedy]


In [61]:
movies_df.tail(10)

,movieId,title,genres
9732,193565,Gintama: The Movie,"[Action, Animation, Comedy, Sci-Fi]"
9733,193567,anohana: The Flower We Saw That Day - The Movie,"[Animation, Drama]"
9734,193571,Silver Spoon,"[Comedy, Drama]"
9735,193573,Love Live! The School Idol Movie,[Animation]
9736,193579,Jon Stewart Has Left the Building,[Documentary]
9737,193581,Black Butler: Book of the Atlantic,"[Action, Animation, Comedy, Fantasy]"
9738,193583,No Game No Life: Zero,"[Animation, Comedy, Fantasy]"
9739,193585,Flint,[Drama]
9740,193587,Bungo Stray Dogs: Dead Apple,"[Action, Animation]"
9741,193609,Andrew Dice Clay: Dice Rules,[Comedy]


As genres in the form of list is not optimal for content based recommender system so we will use One Hot Encoding Technique.
For each movie, we'll assign 1 to the genre type which is present in movie and 0 if it's not in the vector form.

In [63]:
moviesGenres_df = movies_df.copy()
for index,row in movies_df.iterrows():
    for genre in row['genres']:
        moviesGenres_df.at[index,genre] = 1
moviesGenres_df = moviesGenres_df.fillna(0)
moviesGenres_df.head(10)

,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,Heat,"[Action, Crime, Thriller]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,Sabrina,"[Comedy, Romance]",0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,Tom and Huck,"[Adventure, Children]",1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,Sudden Death,[Action],0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,GoldenEye,"[Action, Adventure, Thriller]",1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
ratings_df=ratings_df.drop('timestamp',1)
ratings_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [87]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5},
            {'title':'Heat', 'rating' : 4},
            {'title' :'Sabrina','rating' : 3.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,rating,title
0,5.0,"Breakfast Club, The"
1,3.5,Toy Story
2,2.0,Jumanji
3,5.0,Pulp Fiction
4,4.5,Akira
5,4.0,Heat
6,3.5,Sabrina


In [88]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputId 

,movieId,title,genres
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji,"[Adventure, Children, Fantasy]"
5,6,Heat,"[Action, Crime, Thriller]"
6,7,Sabrina,"[Comedy, Romance]"
257,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]"
697,915,Sabrina,"[Comedy, Romance]"
973,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]"
1445,1968,"Breakfast Club, The","[Comedy, Drama]"


In [89]:
inputMovies = pd.merge(inputId, inputMovies)
inputMovies


,movieId,title,genres,rating
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",3.5
1,2,Jumanji,"[Adventure, Children, Fantasy]",2.0
2,6,Heat,"[Action, Crime, Thriller]",4.0
3,7,Sabrina,"[Comedy, Romance]",3.5
4,915,Sabrina,"[Comedy, Romance]",3.5
5,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",5.0
6,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",4.5
7,1968,"Breakfast Club, The","[Comedy, Drama]",5.0


In [90]:
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('genres', 1)
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,6,Heat,4.0
3,7,Sabrina,3.5
4,915,Sabrina,3.5
5,296,Pulp Fiction,5.0
6,1274,Akira,4.5
7,1968,"Breakfast Club, The",5.0


In [92]:
#Filtering out the movies from the input
userMovies = moviesGenres_df[moviesGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,Heat,"[Action, Crime, Thriller]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,Sabrina,"[Comedy, Romance]",0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
257,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
697,915,Sabrina,"[Comedy, Romance]",0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
973,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1445,1968,"Breakfast Club, The","[Comedy, Drama]",0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
userMovies = userMovies.reset_index(drop=True)
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1)
userGenreTable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


we're going to turn each genre into weights. We can do this by using the input's reviews and multiplying them into the input's genre table and then summing up the resulting table by column. This operation is actually a dot product between a matrix and a vector, so we can simply accomplish by calling Pandas's "dot" function.

In [99]:
ip=inputMovies['rating']
ip

0    3.5
1    2.0
2    4.0
3    3.5
4    3.5
5    5.0
6    4.5
7    5.0
Name: rating, dtype: float64

In [100]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(ip)

userProfile

Adventure             10.0
Animation              8.0
Children               5.5
Comedy                20.5
Fantasy                5.5
Romance                8.5
Drama                  8.5
Action                 8.5
Crime                  7.5
Thriller               7.5
Horror                 0.0
Mystery                0.0
Sci-Fi                 4.5
War                    0.0
Musical                0.0
Documentary            0.0
IMAX                   0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

Now, we have the weights for every of the user's preferences. Using this, we can recommend movies that satisfy the user's preferences.

In [102]:
genreTable = moviesGenres_df.set_index(moviesGenres_df['movieId'])
genreTable.head()

,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,,
1,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Jumanji,"[Adventure, Children, Fantasy]",1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Grumpier Old Men,"[Comedy, Romance]",0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Waiting to Exhale,"[Comedy, Drama, Romance]",0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,Father of the Bride Part II,[Comedy],0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
genreTable = genreTable.drop('movieId',1).drop('title',1).drop('genres',1)
genreTable.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


now, based on user's profile and movie genres ,we'll take weighted average and recommend the most suitable 10 movies according to their interest

In [105]:
recommend_df = ((genreTable*userProfile).sum(axis=1)/userProfile.sum())
recommend_df.head(10)

movieId
1     0.523810
2     0.222222
3     0.306878
4     0.396825
5     0.216931
6     0.248677
7     0.306878
8     0.164021
9     0.089947
10    0.275132
dtype: float64

In [109]:
recommend_df = recommend_df.sort_values(ascending =False)
recommend_df.head()

movieId
4719     0.730159
4956     0.671958
31367    0.661376
81132    0.661376
1907     0.645503
dtype: float64

Final Recommendation of top 10 movies

In [110]:
movies_df.loc[movies_df['movieId'].isin(recommend_df.head(10).keys())]

,movieId,title,genres
1390,1907,Mulan,"[Adventure, Animation, Children, Comedy, Drama..."
3460,4719,Osmosis Jones,"[Action, Animation, Comedy, Crime, Drama, Roma..."
3608,4956,"Stunt Man, The","[Action, Adventure, Comedy, Drama, Romance, Th..."
5774,31367,"Chase, The","[Action, Adventure, Comedy, Crime, Romance, Th..."
6448,51939,TMNT (Teenage Mutant Ninja Turtles),"[Action, Adventure, Animation, Children, Comed..."
7069,69644,Ice Age: Dawn of the Dinosaurs,"[Action, Adventure, Animation, Children, Comed..."
7441,81132,Rubber,"[Action, Adventure, Comedy, Crime, Drama, Film..."
7805,92348,Puss in Boots (Nagagutsu o haita neko),"[Adventure, Animation, Children, Comedy, Fanta..."
8349,108540,Ernest & Célestine (Ernest et Célestine),"[Adventure, Animation, Children, Comedy, Drama..."
8597,117646,Dragonheart 2: A New Beginning,"[Action, Adventure, Comedy, Drama, Fantasy, Th..."
